#### __신경망 모델 구성하기__

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted Class: {y_pred}")

Predicted Class: tensor([6], device='cuda:0')


In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#### __모델 계층(Layer)__
- nn.Flatten
- nn.Linear
- nn.ReLU
- nn.Sequential
- nn.softmax

In [11]:
# 28 * 28의 2D이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
# dim=0의 미니배치 차원은 유지(3)
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [17]:
# 가중치와 편향을 사용하여 입력에 선형 변환을 적용
# (3, 784) @ (28*28, 20) = (3, 20)
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0944,  0.2346,  0.0408, -0.5941,  0.1585, -0.4212, -0.0283, -0.0398,
         -0.4340,  0.0898, -0.0443,  0.7645, -0.3168,  0.4726, -0.7565, -0.0088,
         -0.5171,  0.0783,  0.1184, -0.4563],
        [ 0.3193,  0.4163,  0.1764, -0.6157, -0.3974, -0.2592,  0.0675,  0.1516,
         -0.5632,  0.0526,  0.1635,  0.5133, -0.2641,  0.4136, -0.1766,  0.1664,
         -0.3626,  0.0917, -0.2901,  0.3825],
        [ 0.3524,  0.1812,  0.0287, -0.7026, -0.0668, -0.3724, -0.3340,  0.3003,
         -0.3654,  0.2648,  0.0999,  0.6657, -0.6375,  0.6853, -0.3435,  0.0161,
         -0.3697, -0.0537,  0.1697, -0.0908]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0944, 0.2346, 0.0408, 0.0000, 0.1585, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0898, 0.0000, 0.7645, 0.0000, 0.4726, 0.0000, 0.0000, 0.0000, 0.0783,
         0.1184, 0.0000],
        [0.3193, 0.4163, 0.1764, 0.0000, 0.0000, 0.0000, 0.0675, 0.1516, 0.0000,
         0.0526, 0.1635, 0.5133, 0.0000, 0.4136, 0.00

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [20]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1189, 0.0805, 0.1008, 0.1152, 0.1162, 0.0849, 0.0956, 0.0798, 0.0970,
         0.1113],
        [0.1078, 0.0939, 0.0875, 0.1065, 0.1059, 0.0892, 0.0876, 0.0884, 0.1066,
         0.1267],
        [0.1103, 0.0907, 0.0927, 0.1031, 0.1077, 0.0784, 0.1042, 0.0898, 0.1029,
         0.1202]], grad_fn=<SoftmaxBackward0>)

#### __모델 매개변수__

In [21]:
print(f"Model Structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0176,  0.0029,  0.0152,  ..., -0.0032,  0.0030,  0.0233],
        [ 0.0117, -0.0326,  0.0105,  ...,  0.0041, -0.0015,  0.0140]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0286,  0.0033], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0188, -0.0116,  0.0316,  ...,  0.0053, -0.0397,  0.0266],
        [ 0.0226, -0.0211,  0.0172,  ...,  0.0047, -0.0215, -0.0189]],
       device='cuda:0', grad_fn=<SliceBa